In [1]:
import sys
import numpy as np
import cupy as cp

sys.path.append('../')

import mandala
from mandala import Node
from mandala import Variable

import mandala.autodiff as ad
import mandala.autodiff.functions as F

cp.cuda.Device(1).use()

In [2]:
class Model(ad.Graph):
    def __init__(self):
        super(Model, self).__init__()

        self.l0 = F.Linear(   5, 1000)
        self.l1 = F.Linear(1000, 1000)
        self.l2 = F.Linear(1000, 1000)
        self.l3 = F.Linear(1000, 1000)
        self.l4 = F.Linear(1000, 1000)
        self.l5 = F.Linear(1000, 1000)
        self.l6 = F.Linear(1000,    3)

    def __call__(self, x):
        h = F.relu(self.l0(x))
        h = F.relu(self.l1(h))
        h = F.relu(self.l2(h))
        h = F.relu(self.l3(h))
        h = F.relu(self.l4(h))
        h = F.relu(self.l5(h))
        y = F.relu(self.l6(h))
        return y

In [3]:
xp = cp

In [4]:
model = Model()
model.to_gpu()

In [5]:
W = xp.arange(15, dtype=np.float32).reshape(3, 5)
b = xp.arange(3, dtype=np.float32)

In [6]:
model.subgraphs

{'l0': <mandala.autodiff.functions.linear.Linear at 0x7f6f03705240>,
 'l1': <mandala.autodiff.functions.linear.Linear at 0x7f6f037050f0>,
 'l2': <mandala.autodiff.functions.linear.Linear at 0x7f6f03705400>,
 'l3': <mandala.autodiff.functions.linear.Linear at 0x7f6f03705438>,
 'l4': <mandala.autodiff.functions.linear.Linear at 0x7f6f03705550>,
 'l5': <mandala.autodiff.functions.linear.Linear at 0x7f6f03705748>,
 'l6': <mandala.autodiff.functions.linear.Linear at 0x7f6f03705668>}

## Test

In [7]:
import time

In [8]:
batch_size = 32

In [9]:
model.l1.b.data.device

<CUDA Device 1>

In [ ]:
s = time.time()
lr = 1e-4

for i in range(10000):
    # make batch
    x = Variable(xp.random.random((batch_size, 5)).astype(np.float32))
    t = Variable(xp.matmul(x.data, W.T) + b)
    
    # forward
    y  = model(x)
    loss = (y - t) ** 2 / batch_size

    # loss
    loss = F.sum((y - t) ** 2) / batch_size

    # backward
    model.cleargrads()
    loss.backward()
    
    # update
    for p in model.params.values():
        p.data -= lr * p.grad.data

    #print(loss.data)



print('time:', time.time() - s)

In [ ]:
for param in model.params.values():
    del param.grad

## Chainer との速度比較

In [ ]:
import chainer
import chainer.functions as F
import chainer.links as L
from chainer import optimizer

In [ ]:
cp.cuda.Device(0).use()

In [ ]:
W = xp.arange(15, dtype=np.float32).reshape(3, 5)
b = xp.arange(3, dtype=np.float32)

In [ ]:
class Model(chainer.Chain):
    def __init__(self):
        super().__init__()
        with self.init_scope():
            self.l0 = L.Linear(   5, 1000)
            self.l1 = L.Linear(1000, 1000)
            self.l2 = L.Linear(1000, 1000)
            self.l3 = L.Linear(1000, 1000)
            self.l4 = L.Linear(1000, 1000)
            self.l5 = L.Linear(1000, 1000)
            self.l6 = L.Linear(1000,    3)

    def __call__(self, x):
        h  = F.relu(self.l0(x))
        h  = F.relu(self.l1(h))
        h  = F.relu(self.l2(h))
        h  = F.relu(self.l3(h))
        h  = F.relu(self.l4(h))
        h  = F.relu(self.l5(h))
        y  = self.l6(h)
        return y

In [ ]:
model = Model()
model.to_gpu()

In [ ]:
opt = chainer.optimizers.SGD(lr=1e-4)
opt.setup(model)

In [ ]:
batchsize = 32
s = time.time()

for i in range(10000):
    # make batch
    x = xp.random.random((batchsize, 5)).astype(np.float32)
    t = xp.matmul(x, W.T) + b
    
    # forward
    y = model(x)

    # loss
    loss = F.mean_squared_error(y, t)

    # backward
    model.cleargrads()
    loss.backward()
    
    # update
    opt.update()

    # print(loss.data)

print(time.time() - s)